In [1]:

import time
import re
import os
import tempfile
import requests
import json
import os
import pandas as pd
from google.colab import drive
from datetime import datetime
from openpyxl import load_workbook
from calendar import monthcalendar
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
drive.mount('/content/drive')
import calendar
from datetime import datetime, timedelta
from collections import OrderedDict

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def get_food_orders_summary(month, year, email, password):
    food = OrderedDict()
    # Login and get token
    login_url = "https://admin.nompenang.com/auth/login"
    payload = {"email": email, "password": password}

    session = requests.Session()
    response = session.post(login_url, json=payload)
    token = response.json()['data']['access_token']

    # Fetch orders
    orders_url = "https://admin.nompenang.com/items/orders"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "fields": "id,order_number,customer_name,customer_phone,delivery_method,status,date_created,schedule_date,schedule_time,total_amount,payment_method,delivery_address",
        "limit": 500
    }

    response = session.get(orders_url, headers=headers, params=params)
    data = response.json()
    orders = data['data']

    # Filter completed orders for the specified date
    completed_order_numbers = [
      order for order in orders
      if datetime.strptime(order['schedule_date'], '%Y-%m-%d').month == month and
        datetime.strptime(order['schedule_date'], '%Y-%m-%d').year == year and
        order['status'] != 'cancelled']

    items_id= [order['id'] for order in completed_order_numbers]
    # Fetch order items
    item_url = "https://admin.nompenang.com/items/order_items"
    item_params = {
        "fields": "id,item_name,quantity,total_price",
        "filter": f'{{"order_id":{{"_in":{items_id}}}}}',
        "limit": 500
    }

    item_response = session.get(item_url, headers=headers, params=item_params)
    item_data = item_response.json()
    items = item_data['data']

    # Count food items
    for item in items:
        item_name = item['item_name']
        quantity = item['quantity']

        if item_name in food:
            food[item_name] += quantity
        else:
            food[item_name] = quantity

    # Convert to lists for return
    food_items = list(food.keys())
    quantities = list(food.values())

    return food_items, quantities



['Nasi Lemak Lovers', 'Char Koay Teow + Chicken Satay + Bur Bur Cha Cha', 'KL Hokkien Char', 'Penang Assam Laksa', 'Beef Rendang with Nasi Lemak', 'Nasi Lemak combo with Beef Rendang and fried Chicken', 'Pandan Kaya (Coconut Jam)', 'Ondeh Ondeh', 'Kuih Ketayap (香兰椰丝卷)', 'Pulut Inti', 'Roti Canai with Curry sauce', 'Nasi Lemak with Fried Chicken + Chicken Satay + Bur Bur Cha Cha', 'Mamak Teh Tarik', 'Penang Char Koay Teow', 'Penang Hainanese Chicken Rice', 'Nasi Lemak with Sambal Seafood ', 'Malaysian Fried Chicken Wings (槟城炸鸡翅)', 'Chicken Satay', 'Sambal Okra and Eggplant', 'Wah Tan Hor', 'Sambal Kang Kung', 'Curry Laksa', 'Curry Chicken with Nasi Lemak', 'Pulut Tai Tai (3 piece)', 'Nasi Lemak Combo with Chicken Curry and Fried Chicken', 'Mee Goreng', 'Sambal Three King with Shrimp and Calamari', 'Chicken Curry Puff', 'Ang Ku Kuih', 'Ikan Bakar (Grilled Fish) with stream rice + Malaysian Fried Chicken Wings + Pulut Inti ', 'Ikan Bakar (Grilled Fish)', 'Peanut Cookies (贡糖)', 'Bah Kut Te

In [25]:
def workbook (year, month, email, password):

  workbook = load_workbook('/content/drive/MyDrive/Colab_Notebooks/Sales Data.xlsx')
  center_align = Alignment(horizontal='center', vertical='center', wrap_text=True) #Alignment for Excel
  full_month_name = calendar.month_name[month] # Covert int month to str Month
  sheet_name = f"{full_month_name} Item Sales" # Name of the Excel Worksheet

  if sheet_name in workbook.sheetnames: # Select worksheet if exists

      ws = workbook[f"{full_month_name} Item Sales"]

  else:        # Else create new sheet

      ws = workbook.create_sheet(f"{full_month_name} Item Sales")

  ws['A1'] = f"{full_month_name}" # title for Table
  ws['A1'].alignment = center_align
  ws['A1'].font = Font(bold=True)
  ws.merge_cells('A1:B1')
  current_row = 2 # Start row for Headers
  current_week = 1 # Start of the month starts on Week 1
  header = ['Items','Quantity']

  for col, header in enumerate(header, 1): #start column count at 1 because excel counts starting at 1 instead of 0

          cell = ws.cell(row = current_row, column = col, value = header)
          cell.alignment = center_align

  current_row += 1  # One row after header
  food_items, quantities = get_food_orders_summary(month, year, email, password)
  food_items_sorted, quantities_sorted = zip(*sorted(zip(food_items, quantities),
                                                     key=lambda x: x[1], reverse=True)) #Sort list before putting in excel because openpxyl does not have a native sort function

  for item, quantity in zip(food_items_sorted, quantities_sorted):

      item_cell = ws.cell(row=current_row, column=1, value=item)
      item_cell.alignment = center_align
      quantity_cell = ws.cell(row=current_row, column=2, value=quantity)
      quantity_cell.alignment = center_align
      current_row += 1

  total_cell = ws.cell(row=current_row, column=1)
  total_cell.alignment = center_align
  total_cell.value = "Total"
  total_quantities_cell = ws.cell(row=current_row, column=2)
  total_quantities_cell.alignment = center_align
  total_quantities_cell.value = sum(quantities_sorted)
  ws.column_dimensions['A'].width = 30

  return workbook


In [26]:
month = 10
year = 2025
email= '#####' #redacted
password= '######' #redacted
workbook = workbook(year, month, email, password)
workbook.save('/content/temp_sales_data.xlsx')
!cp '/content/temp_sales_data.xlsx' '/content/drive/MyDrive/Colab_Notebooks/Sales Data.xlsx' #bash
print("File saved successfully!")

Descending list1: ('Penang Hainanese Chicken Rice', 'Penang Char Koay Teow', 'Chicken Curry Puff', 'Ondeh Ondeh', 'Kuih Ketayap (香兰椰丝卷)', 'Penang Spicy Prawn Noodles', 'Wah Tan Hor', 'Ang Ku Kuih (Yellow Mung Bean)', 'Nyoya Pie Tee Cup', 'Roti Canai with Curry sauce', 'Penang Wontan Mee', 'Malaysian Fried Chicken Wings (槟城炸鸡翅)', 'Ikan Bakar (Grilled Fish) with steam rice + Malaysian Fried Chicken Wings + Pulut Inti ', 'Penang Assam Laksa', 'Chicken Satay', 'Ang Ku Kuih', 'Assorted Ku Kuih', 'Green Ku Kuih (Grated Coconut)', 'Mee Goreng', 'Curry Laksa with steam rice + Malaysian Fried Chicken Wings + Pulut Inti', 'KL Hokkien Char', 'Nasi Lemak combo with Beef Rendang and fried Chicken', 'Nasi Lemak with Sambal Seafood ', 'Sambal Kang Kung', 'Penang Koay Teow Theng', 'Beef Rendang with Nasi Lemak', 'Curry Laksa', 'Bah Kut Teh with steam rice +  Malaysian Fried Chicken Wings + Pulut Inti', 'Mamak Teh Tarik', 'Sambal Okra and Eggplant', 'Nasi Lemak Combo with Chicken Curry and Fried Chicke